In [1]:
import numpy as np 
import pandas as pd 
import random
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
import time
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import requests
import subprocess
import re
from tqdm import tqdm
tqdm.pandas()

import os
os.chdir(os.path.abspath(os.path.join(os.getcwd(), "..", "..")))

In [2]:
from utils.web_scrapping import random_scroll,scroll_to_element,scroll_to_bottom
from logger import logger

In [3]:
def launch_chrome_with_debugging(url):
    chrome_path = r"C:\Program Files\Google\Chrome\Application\chrome.exe"
    # user_data_dir = r"C:\\Users\\Harish raju\\AppData\\Local\\Google\\Chrome\\User Data\\Default"  # Adjust this path as needed
    remote_debugging_port = 9222

    # Command to start Chrome with remote debugging enabled
    command = [
        chrome_path,
        f"--remote-debugging-port={remote_debugging_port}",
        # f"--user-data-dir={user_data_dir}",
        url
    ]

    # Launch Chrome
    subprocess.Popen(command)

url = "https://www.imdb.com/search/title/?groups=top_1000&count=100&sort=num_votes,desc"
# Launch Chrome with the target URL
launch_chrome_with_debugging(url)

# Wait for a few seconds to ensure Chrome has started
time.sleep(5)


In [4]:
# Function to get the WebSocket Debugger URL
def get_websocket_debugger_url():
    response = requests.get('http://localhost:9222/json')
    if response.status_code == 200:
        json_data = response.json()
        for page in json_data:
            if "imdb" in page['url']:  # Match any Amazon page
                return page['webSocketDebuggerUrl']
    return None

# Check if Chrome is running with remote debugging
websocket_debugger_url = get_websocket_debugger_url()
if not websocket_debugger_url:
    raise Exception("Could not get the WebSocket debugger URL. "
                    "Make sure Chrome is running with remote debugging enabled.")

# Set up Selenium to use the existing Chrome instance
chrome_options = Options()
chrome_options.debugger_address = "localhost:9222"

# Set a custom user agent (optional)
user_agent = ("Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
              "AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")
chrome_options.add_argument(f"user-agent={user_agent}")

# Initialize WebDriver
driver = webdriver.Chrome(options=chrome_options)

In [5]:
driver.get(url)

time.sleep(3)

In [6]:
# Constants
MAX_REVIEWS = 1000
SCROLLING_PERCENTAGE = 0.95
WAIT_TIME = 5  # Time to wait after scrolling or clicking


def scroll_to_bottom():
    """Scrolls to the bottom of the page."""
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)  # Allow time for content to load

def scroll_to_element(element):
    """Scrolls smoothly to the given element."""
    driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'center'});", element)
    time.sleep(2)  # Small delay for UI adjustment

def click_button():
    """Finds and clicks the 'See More' button, if available."""
    try:
        button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//button[contains(@class, 'ipc-see-more__button')]"))
        )
        scroll_to_element(button)
        driver.execute_script("arguments[0].click();", button)  # JavaScript click to avoid interception issues
        print("Clicked 'See More' button.")
        time.sleep(WAIT_TIME)  # Allow time for new content to load
    except Exception as e:
        print("No more 'See More' button found or an error occurred:", e)
        return False  # Stop execution if button isn't found
    return True

def collect_reviews():
    """Collects the review elements from the page."""
    return driver.find_elements(By.CLASS_NAME, "ipc-metadata-list-summary-item")

# Main loop
while True:
    # print("Scrolling down...")
    # scroll_to_bottom()
    
    print("Attempting to click 'See More' button...")
    if not click_button():  # If button is not found, exit loop
        break

    titles = collect_reviews()
    print(f"Total Reviews Collected: {len(titles)}")

    if len(titles) >= MAX_REVIEWS:
        break

print("Review collection complete.")


Attempting to click 'See More' button...
Clicked 'See More' button.
Total Reviews Collected: 200
Attempting to click 'See More' button...
Clicked 'See More' button.
Total Reviews Collected: 300
Attempting to click 'See More' button...
Clicked 'See More' button.
Total Reviews Collected: 400
Attempting to click 'See More' button...
Clicked 'See More' button.
Total Reviews Collected: 500
Attempting to click 'See More' button...
Clicked 'See More' button.
Total Reviews Collected: 600
Attempting to click 'See More' button...
Clicked 'See More' button.
Total Reviews Collected: 700
Attempting to click 'See More' button...
Clicked 'See More' button.
Total Reviews Collected: 800
Attempting to click 'See More' button...
Clicked 'See More' button.
Total Reviews Collected: 900
Attempting to click 'See More' button...
Clicked 'See More' button.
Total Reviews Collected: 1000
Review collection complete.


In [7]:
# import re
# from tqdm import tqdm
# from selenium.webdriver.common.by import By

# # Dictionary to store movie titles and their corresponding IMDb links
# movie_data = {}

# # Iterate through each extracted 'li' element containing movie details
# # for movie_element in tqdm(titles, desc='Processing movie titles'):
# for movie_element in titles:

#     # Extract the movie title from the 'h3' tag inside the element
#     raw_title = movie_element.find_element(By.XPATH, ".//h3[contains(@class, 'ipc-title__text')]").text
#     print(f"extracting {raw_title}")

#     # Remove leading numbers (e.g., "1. The Shawshank Redemption" → "The Shawshank Redemption")
#     movie_title = re.sub(r"^\d+\.\s*", "", raw_title)
#     p

#     # Extract the hyperlink to the movie's IMDb page
#     movie_link = movie_element.find_element(By.TAG_NAME, 'a').get_attribute('href')

#     # Store the extracted data in the dictionary
#     movie_data[movie_title] = movie_link  # Corrected key to store the title properly

# # The 'movie_data' dictionary now contains {movie_title: IMDb_link} pairs


In [8]:
# List to store movie titles and their corresponding IMDb links
movie_data = []

# Iterate through each extracted 'li' element containing movie details
for movie_element in titles:
    # Extract the movie title from the 'h3' tag inside the element
    raw_title = movie_element.find_element(By.XPATH, ".//h3[contains(@class, 'ipc-title__text')]").text
    print(f"Extracting: {raw_title}")

    # Remove leading numbers (e.g., "1. The Shawshank Redemption" → "The Shawshank Redemption")
    movie_title = re.sub(r"^\d+\.\s*", "", raw_title)

    # Extract the hyperlink to the movie's IMDb page
    movie_link = movie_element.find_element(By.TAG_NAME, 'a').get_attribute('href')

    # Append data as a tuple (no risk of overwriting)
    movie_data.append((movie_title, movie_link))

# Convert list to DataFrame
df_movies = pd.DataFrame(movie_data, columns=["Title", "IMDb Link"])

# Display the number of rows
print(f"Total movies extracted: {len(df_movies)}")


Extracting: 1. The Shawshank Redemption
Extracting: 2. The Dark Knight
Extracting: 3. Inception
Extracting: 4. Fight Club
Extracting: 5. Forrest Gump
Extracting: 6. Pulp Fiction
Extracting: 7. Interstellar
Extracting: 8. The Matrix
Extracting: 9. The Godfather
Extracting: 10. The Lord of the Rings: The Fellowship of the Ring
Extracting: 11. The Lord of the Rings: The Return of the King
Extracting: 12. The Dark Knight Rises
Extracting: 13. Se7en
Extracting: 14. The Lord of the Rings: The Two Towers
Extracting: 15. Django Unchained
Extracting: 16. Gladiator
Extracting: 17. The Wolf of Wall Street
Extracting: 18. Inglourious Basterds
Extracting: 19. Batman Begins
Extracting: 20. The Silence of the Lambs
Extracting: 21. Joker
Extracting: 22. Saving Private Ryan
Extracting: 23. Shutter Island
Extracting: 24. Schindler's List
Extracting: 25. The Prestige
Extracting: 26. The Avengers
Extracting: 27. Star Wars: Episode IV - A New Hope
Extracting: 28. The Departed
Extracting: 29. The Green Mile

In [10]:
df_movies

,Title,IMDb Link
0,The Shawshank Redemption,https://www.imdb.com/title/tt0111161/?ref_=sr_i_1
1,The Dark Knight,https://www.imdb.com/title/tt0468569/?ref_=sr_i_2
2,Inception,https://www.imdb.com/title/tt1375666/?ref_=sr_i_3
3,Fight Club,https://www.imdb.com/title/tt0137523/?ref_=sr_i_4
4,Forrest Gump,https://www.imdb.com/title/tt0109830/?ref_=sr_i_5
...,...,...
995,Brother,https://www.imdb.com/title/tt0118767/?ref_=sr_...
996,The Twilight Samurai,https://www.imdb.com/title/tt0351817/?ref_=sr_...
997,Nayakan,https://www.imdb.com/title/tt0093603/?ref_=sr_...
998,That Obscure Object of Desire,https://www.imdb.com/title/tt0075824/?ref_=sr_...


In [11]:
df_movies['Title'].value_counts()

Title
The Girl with the Dragon Tattoo    2
All Quiet on the Western Front     2
Scarface                           2
Beauty and the Beast               2
Drishyam                           2
                                  ..
Brother                            1
The Twilight Samurai               1
Nayakan                            1
That Obscure Object of Desire      1
Great Expectations                 1
Name: count, Length: 994, dtype: int64

In [12]:
pwd

'c:\\Users\\Harish Raju\\Desktop\\projects\\IMDB'

In [13]:
df_movies.to_csv('web_scrapper/data/Movies_index.csv',index=False)